In [1]:
import numpy as np
import pandas as pd
import sys
import os 
import warnings
from datetime import datetime
from src.logger import logging
from src.exception import CustomException
warnings.filterwarnings("ignore")

In [2]:
## columns needed from icu stay csv
icu_keys = ['HADM_ID', 'ICUSTAY_ID', 'SUBJECT_ID', 'INTIME', 'OUTTIME', 'LOS']
icu_dates = ['INTIME', 'OUTTIME']

## columns needed from admissions csv
admissions_keys = ["HADM_ID", "ADMITTIME", "DEATHTIME", "ADMISSION_LOCATION",
                   "INSURANCE", "LANGUAGE", "RELIGION", "MARITAL_STATUS", "ETHNICITY",
                   "DIAGNOSIS","HAS_CHARTEVENTS_DATA"]
admissions_dates = ["ADMITTIME","DEATHTIME"]

## columns needed for patients csv
patients_keys = ['SUBJECT_ID', 'GENDER']

## columns needed from chartevents csv
chart_events_keys = ['ICUSTAY_ID', 'ITEMID', 'VALUENUM', 'CHARTTIME']
chart_events_dates = ['CHARTTIME']


In [3]:
logging.info('Load the Necessory Datasets...')
icu_stay_path=r'D:\FINALYEARPROJECTREC\data\ICUSTAYS.csv'
icu_stays = pd.read_csv(icu_stay_path, usecols=icu_keys, parse_dates=icu_dates)

In [4]:
admissions_path=r'D:\FINALYEARPROJECTREC\data\ADMISSIONS.csv'
admissions = pd.read_csv(admissions_path, usecols=admissions_keys, parse_dates=admissions_dates)

In [5]:
logging.info('left join icu_stays, admissions data tables, based on HADM_ID')
icu_adm = pd.merge(icu_stays, admissions, on="HADM_ID", how="left")
print("ICU Adm shape: ", icu_adm.shape)


ICU Adm shape:  (61532, 16)


In [6]:
icu_adm.head()

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,INTIME,OUTTIME,LOS,ADMITTIME,DEATHTIME,ADMISSION_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,DIAGNOSIS,HAS_CHARTEVENTS_DATA
0,268,110404,280836,2198-02-14 23:27:00,2198-02-18 05:26:00,3.2490,11-02-2198 13:40,2198-02-18 03:55:00,EMERGENCY ROOM ADMIT,Medicare,NaN,CATHOLIC,SEPARATED,HISPANIC OR LATINO,DYSPNEA,1
1,269,106296,206613,2170-11-05 11:05:00,2170-11-08 17:46:00,3.2788,05-11-2170 11:04,NaT,EMERGENCY ROOM ADMIT,Medicaid,NaN,UNOBTAINABLE,SINGLE,WHITE,SEPSIS;PILONIDAL ABSCESS,1
2,270,188028,220345,2128-06-24 15:05:00,2128-06-27 12:32:00,2.8939,23-06-2128 18:26,NaT,PHYS REFERRAL/NORMAL DELI,Medicare,NaN,JEHOVAH'S WITNESS,MARRIED,UNKNOWN/NOT SPECIFIED,CAROTID STENOSIS\CAROTID ANGIOGRAM AND STENT,1
3,271,173727,249196,2120-08-07 23:12:00,2120-08-10 00:39:00,2.0600,07-08-2120 18:56,NaT,TRANSFER FROM HOSP/EXTRAM,Private,ENGL,NOT SPECIFIED,MARRIED,PATIENT DECLINED TO ANSWER,GALLSTONE PANCREATITIS,1
4,272,164716,210407,2186-12-25 21:08:00,2186-12-27 12:01:00,1.6202,25-12-2186 21:06,NaT,TRANSFER FROM HOSP/EXTRAM,Medicare,NaN,UNOBTAINABLE,MARRIED,WHITE,PULMONARY EMBOLIS,1


In [7]:
logging.info('only keep rows that have chart events data')
icu_adm = icu_adm.loc[(icu_adm["HAS_CHARTEVENTS_DATA"] == 1)]
print(icu_adm.shape)

(61051, 16)


In [8]:
logging.info('show detailed dataframe info without abbreviation')
pd.set_option('display.max_columns', None)

In [9]:
logging.info(" read patients csv, and save into dataframe")
patients_path='D:\FINALYEARPROJECTREC\data\PATIENTS.csv'
patients = pd.read_csv(patients_path, usecols=patients_keys)
icu_adm = pd.merge(icu_adm, patients, on='SUBJECT_ID', how="left")


In [10]:
icu_adm.head()

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,INTIME,OUTTIME,LOS,ADMITTIME,DEATHTIME,ADMISSION_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,DIAGNOSIS,HAS_CHARTEVENTS_DATA,GENDER
0,268,110404,280836,2198-02-14 23:27:00,2198-02-18 05:26:00,3.2490,11-02-2198 13:40,2198-02-18 03:55:00,EMERGENCY ROOM ADMIT,Medicare,NaN,CATHOLIC,SEPARATED,HISPANIC OR LATINO,DYSPNEA,1,F
1,269,106296,206613,2170-11-05 11:05:00,2170-11-08 17:46:00,3.2788,05-11-2170 11:04,NaT,EMERGENCY ROOM ADMIT,Medicaid,NaN,UNOBTAINABLE,SINGLE,WHITE,SEPSIS;PILONIDAL ABSCESS,1,M
2,270,188028,220345,2128-06-24 15:05:00,2128-06-27 12:32:00,2.8939,23-06-2128 18:26,NaT,PHYS REFERRAL/NORMAL DELI,Medicare,NaN,JEHOVAH'S WITNESS,MARRIED,UNKNOWN/NOT SPECIFIED,CAROTID STENOSIS\CAROTID ANGIOGRAM AND STENT,1,M
3,271,173727,249196,2120-08-07 23:12:00,2120-08-10 00:39:00,2.0600,07-08-2120 18:56,NaT,TRANSFER FROM HOSP/EXTRAM,Private,ENGL,NOT SPECIFIED,MARRIED,PATIENT DECLINED TO ANSWER,GALLSTONE PANCREATITIS,1,F
4,272,164716,210407,2186-12-25 21:08:00,2186-12-27 12:01:00,1.6202,25-12-2186 21:06,NaT,TRANSFER FROM HOSP/EXTRAM,Medicare,NaN,UNOBTAINABLE,MARRIED,WHITE,PULMONARY EMBOLIS,1,M


In [11]:
logging.info("initialize chartevents column with empty array for each row")
icu_adm['CHARTEVENTS'] = np.empty((len(icu_adm), 0)).tolist()

logging.info("## set label to 1 if 1) death time exists 2) death time is between in time and out time of icu stay, else label 0")
icu_adm['LABEL'] = np.where(((pd.notna(icu_adm['DEATHTIME'])) & (icu_adm['INTIME'] <= icu_adm['DEATHTIME']) & (icu_adm['DEATHTIME'] <= icu_adm['OUTTIME'])), 1, 0)

logging.info("convert icu stay dataframe to dictionary format")
icu_adm_dict = icu_adm.to_dict('records')

In [12]:
icu_adm_dict

[{'SUBJECT_ID': 268,
  'HADM_ID': 110404,
  'ICUSTAY_ID': 280836,
  'INTIME': Timestamp('2198-02-14 23:27:00'),
  'OUTTIME': Timestamp('2198-02-18 05:26:00'),
  'LOS': 3.249,
  'ADMITTIME': '11-02-2198 13:40',
  'DEATHTIME': Timestamp('2198-02-18 03:55:00'),
  'ADMISSION_LOCATION': 'EMERGENCY ROOM ADMIT',
  'INSURANCE': 'Medicare',
  'LANGUAGE': nan,
  'RELIGION': 'CATHOLIC',
  'MARITAL_STATUS': 'SEPARATED',
  'ETHNICITY': 'HISPANIC OR LATINO',
  'DIAGNOSIS': 'DYSPNEA',
  'HAS_CHARTEVENTS_DATA': 1,
  'GENDER': 'F',
  'CHARTEVENTS': [],
  'LABEL': 1},
 {'SUBJECT_ID': 269,
  'HADM_ID': 106296,
  'ICUSTAY_ID': 206613,
  'INTIME': Timestamp('2170-11-05 11:05:00'),
  'OUTTIME': Timestamp('2170-11-08 17:46:00'),
  'LOS': 3.2788,
  'ADMITTIME': '05-11-2170 11:04',
  'DEATHTIME': NaT,
  'ADMISSION_LOCATION': 'EMERGENCY ROOM ADMIT',
  'INSURANCE': 'Medicaid',
  'LANGUAGE': nan,
  'RELIGION': 'UNOBTAINABLE',
  'MARITAL_STATUS': 'SINGLE',
  'ETHNICITY': 'WHITE',
  'DIAGNOSIS': 'SEPSIS;PILONIDAL A

In [13]:
# jgc

In [ ]:
logging.info("read chart events in chunks form and append chart events to matching icu stay id (3hrs from in_time)")
chunk_size=10000000
chart_events_path=r'D:\FINALYEARPROJECTREC\data\CHARTEVENTS.csv'
with pd.read_csv(chart_events_path, usecols=chart_events_keys, parse_dates=chart_events_dates, chunksize=chunk_size) as reader:
    count=0
    for chunk in reader:
        s=datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        # convert null to None (valuenum)
        # chunk = chunk.replace({np.nan: None})
        # eliminate nan value_nums in CHARTEVENTS.csv
        chunk = chunk[chunk['VALUENUM'].notna()]
        print(chunk.head())
        # logging.info(" create 'ITEMID', 'VALUENUM', 'CHARTTIME' of each row as single list, and add into new column 'CHARTEVENTS'")
        chunk['CHARTEVENTS'] = chunk[chunk.columns[1:]].apply(lambda x: list(x), axis=1)
        # drop previous 'ITEMID', 'VALUENUM', 'CHARTTIME' columns
        chunk.drop(['ITEMID', 'CHARTTIME', 'VALUENUM'], axis=1, inplace=True)
        # group chunk dataframe by ICUSTAY_ID, and concat all chart events data into single list
        chunk = chunk.groupby('ICUSTAY_ID')['CHARTEVENTS'].apply(list).reset_index(name='CHARTEVENTS')
        # convert dataframe to dictionary format
        chunk_dict = chunk.to_dict('records')
    
    # for each ICUSTAY_ID in chart events chunk
        for i in chunk_dict:
            icu_id = i['ICUSTAY_ID']
            r = 0
            # loop through icu stays dataframe
            for j in icu_adm_dict:
                # if match exists between chunk ICUSTAY_ID and icu stays dataframe ICUSTAY_ID
                if j['ICUSTAY_ID'] == icu_id:
                    # if chart events of icu stay is equal to 100, break
                    if len(j['CHARTEVENTS']) == 100:
                        r += 1
                        break

                    # access in time of icu stay
                    in_time = j['INTIME']

                    # for each chunk CHARTEVENTS data
                    for k in i['CHARTEVENTS']:
                        # if chart event time is within 3 hours after in time
                        if (k[1] >= in_time) & ((k[1] - in_time)/np.timedelta64(1, 'h') <= 3):
                            # modify timestamp to (chart event time - in time) in minutes
                            k[1] = (k[1] - in_time)/np.timedelta64(1, 'm')
                            # append icu stays dataframe CHARTEVENTS list with chunk CHARTEVENTS data
                            # order: timestamp, item_id, value_num
                            j['CHARTEVENTS'].append([k[1], k[0], k[2]])
                        # if chart events of icu stay length is equal to 100, break
                        if len(j['CHARTEVENTS']) == 100:
                            r += 1
                            break

        s = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        print("[", s, "] FINISHED READING CHARTEVENTS CHUNK ", count)
        count += 1

   ICUSTAY_ID  ITEMID           CHARTTIME  VALUENUM
0    241249.0  223834 2134-05-12 12:00:00     15.00
1    241249.0  223835 2134-05-12 12:00:00    100.00
2    241249.0  224328 2134-05-12 12:00:00      0.37
3    241249.0  224329 2134-05-12 12:00:00      6.00
4    241249.0  224330 2134-05-12 12:00:00      2.50
[ 2025-02-28 16:14:17 ] FINISHED READING CHARTEVENTS CHUNK  0
          ICUSTAY_ID  ITEMID           CHARTTIME  VALUENUM
10000000    246813.0  220052 2186-10-14 03:57:00      80.0
10000001    246813.0  220074 2186-10-14 03:57:00      15.0
10000002    246813.0  220210 2186-10-14 03:57:00      17.0
10000003    246813.0  220277 2186-10-14 03:57:00     100.0
10000004    246813.0  220292 2186-10-14 03:57:00       4.0


In [ ]:
## save icu stays with chart events to csv file
icu_chart_events = pd.DataFrame.from_records(icu_adm_dict)
print(icu_chart_events.head())
icu_chart_events.to_csv("D:\FINALYEARPROJECTREC\artifacts\icu_with_chart_events_v_not_nan.csv", header=True, index=False)